In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import networkx as nx
import pandas as pd
from datetime import datetime
import math


project = 'asymmetric-cove-187011' # Project ID inserted based on the query results selected to explore
# project = 'glossy-odyssey-366820'
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Farcaster Input Data

In [ ]:
verifications_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/verifications/verifications.parquet')

In [ ]:
# verifications_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/verifications/verifications.parquet')
# profiles_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/profiles/profiles.parquet')
#link_210722_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/links/parquet/links_2021-07-22.parquet')
#reactions_210924_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/reactions/parquet/reactions_2021-09-24.parquet')
links_df = pd.read_parquet('/content/drive/My Drive/farcaster_bigquery/links/parquet/links_followsOnly.parquet')

In [ ]:
# profiles_df.sort_values(by='fid').head()

In [ ]:
# profiles_df[profiles_df.fid == 1]

In [ ]:
links_df.head()

,fid,timestamp,target_fid,type,deleted_at
0,19099,2024-04-20 22:06:08+00:00,461009,follow,NaT
1,252634,2024-05-03 10:06:31+00:00,250833,follow,NaT
2,418937,2024-04-06 09:26:16+00:00,415186,follow,NaT
3,385938,2024-04-17 19:43:30+00:00,403156,follow,NaT
4,500406,2024-04-29 11:21:25+00:00,484567,follow,NaT


In [ ]:
links_df = links_df[links_df.deleted_at.isna()]

In [ ]:
followees_df = links_df.fid.value_counts()

In [ ]:
followers_df = links_df.target_fid.value_counts()

In [ ]:
follows_df = pd.DataFrame({'followers': followers_df, 'followees': followees_df})

In [ ]:
follows_df = follows_df.fillna(1)

In [ ]:
follows_df.iloc[50:60]

,followers,followees
51,7409,611
52,986,187
53,32940,431
54,168721,427
55,66,1
56,266255,2103
57,3123,100
58,1931,760
59,6587,367
60,76306,364


# Process links data (only follows)

In [ ]:
# links_df = links_df[links_df.type.isin(['follow', 'unfollow'])]

In [ ]:
# links_df = links_df.drop_duplicates(['target_fid', 'fid'], keep=False)

In [ ]:
# links_df.to_csv(f'/content/drive/My Drive/farcaster_bigquery/links/csv/links_followsOnly.csv')
# links_df.to_parquet(f'/content/drive/My Drive/farcaster_bigquery/links/parquet/links_followsOnly.parquet')

# Usermass Calculations

In [ ]:
# Variable Coefficients
currentTimestamp = datetime.now()
constantDivisionFactorGt_threshFriendsToFollowersRatioUMass = 5.0
threshAbsNumFriendsUMass = 500
threshFriendsToFollowersRatioUMass = 0.6
deviceWeightAdditive = 0.5
ageWeightAdditive = 0.2
restrictedWeightMultiplicative = 0.1

In [ ]:
# To Calculate

# user = combined users? multiple addresses to a FID?
# userID = FID
# userAge = daysSinceCreation
# isRestricted = reviewVerificationDeletion?
# isSuspended = review?
# isVerified = reviewVerificationsDF?
# hasValidDevice??
# numFollowers = linkTypeFollowersAtDate
# numFollowings = linkTypeFollowingAtDate

In [ ]:
follows_df.to_parquet(f'/content/drive/My Drive/farcaster_bigquery/links/follows_mass.parquet')
follows_df.to_csv(f'/content/drive/My Drive/farcaster_bigquery/links/follows_mass.csv')

In [ ]:
for fid in follows_df.index:
# for fid in range(1,10):
  # print(fid)
  if fid in verifications_df.fid.values:
    # print(f'{fid} is verified')
    userMass = 100
  else:
    userMass = 45

  numFollowings = follows_df.loc[fid, 'followees']
  # print(f'{fid} has {numFollowings} followings')
  numFollowers = follows_df.loc[fid, 'followers']
  # print(f'{fid} has {numFollowers} followers')
  friendsToFollowersRatio = numFollowings / numFollowers
  # print(f'{fid} has a friends to followers ratio of {friendsToFollowersRatio}')
  # if followers > 500 and followings/followers > .6
  if (numFollowers > threshAbsNumFriendsUMass) & (friendsToFollowersRatio > threshFriendsToFollowersRatioUMass):

    # print(constantDivisionFactorGt_threshFriendsToFollowersRatioUMass)
    # print((friendsToFollowersRatio - threshFriendsToFollowersRatioUMass))
    # exped = (constantDivisionFactorGt_threshFriendsToFollowersRatioUMass * round(friendsToFollowersRatio - threshFriendsToFollowersRatioUMass))
    # print(exped)
    adjustedMass = userMass / math.exp((constantDivisionFactorGt_threshFriendsToFollowersRatioUMass * (friendsToFollowersRatio - threshFriendsToFollowersRatioUMass)))
  else:
    adjustedMass = userMass

  follows_df.loc[fid, 'adjustedMass'] = adjustedMass


KeyboardInterrupt: 

In [ ]:
follows_df.head(50)

,followers,followees,adjustedMass
1,15998,2,100.0
2,254609,1197,100.0
3,292760,3083,100.0
4,751,290,45.0
5,1148,190,100.0
6,645,131,45.0
7,485,60,45.0
8,267223,295,100.0
9,894,442,100.0
10,1893,69,45.0


In [ ]:
# if (userId == 0L || user.map(_.safety).exists(_.deactivated)) {
#       None
#     } else {
#       val mass =
#         if (isSuspended)
#           0
#         else if (isVerified)
#           100
#         else {
#           var score = deviceWeightAdditive * 0.1 +
#             (if (hasValidDevice) deviceWeightAdditive else 0)
#           val normalizedAge = if (age > 30) 1.0 else (1.0 min scala.math.log(1.0 + age / 15.0))
#           score *= normalizedAge
#           if (score < 0.01) score = 0.01
#           if (isRestricted) score *= restrictedWeightMultiplicative
#           score = (score min 1.0) max 0
#           score *= 100
#           score
#         }

#       val friendsToFollowersRatio = (1.0 + numFollowings) / (1.0 + numFollowers)
#       val adjustedMass =
#         if (numFollowings > threshAbsNumFriendsUMass &&
#           friendsToFollowersRatio > threshFriendsToFollowersRatioUMass) {
#           mass / scala.math.exp(
#             constantDivisionFactorGt_threshFriendsToFollowersRatioUMass *
#               (friendsToFollowersRatio - threshFriendsToFollowersRatioUMass)
#           )
#         } else {
#           mass
#         }

#       Some(UserMassInfo(userId, adjustedMass))
#     }

In [ ]:
score = deviceWeightAdditive * .1
normalizedAge = 1


In [ ]:
score

0.05

In [ ]:
score * normalizedAge * 100

5.0